In [11]:
import numpy as np
from external_tools.python_octree import octree_to_voxel_grid, import_ot
from tf_lab.voxels.soup import plot_isosurface, uniform_sampling_of_voxels
from geo_tool import Point_Cloud

In [2]:
%matplotlib inline

In [8]:
import os.path as osp
from multiprocessing import Pool
# from functools import partial
# from multiprocessing.dummy import Pool
# from subprocess import call
from general_tools.in_out.basics import files_in_subdirs

In [98]:
# Parameters
n_threads = 30
n_pc_samples = 2048
top_oct_dir = '/data/kaichun/shapeGen/ogn_exps/ogn-autoencoder/panos_decoded_chair_output/'
top_out_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/octree_based/gmm/'
# top_out_dir = osp.join(top_out_dir, str(n_pc_samples) + osp.sep)
# copy_folder_structure(top_mesh_dir, top_out_dir)

In [9]:
ending = '.ot'
search_pattern = ending + '$'
all_in_files = [f for f in files_in_subdirs(top_oct_dir, search_pattern)]
print len(all_in_files)

5712


In [41]:
# def oct_tree_to_pc(ot_file, oct_res, n_pc_points, out_f):
#     ot_struct, _ = import_ot(ot_file)
#     voxels = octree_to_voxel_grid(ot_struct, oct_res)
# #     .astype(np.bool)
# #     voxels = np.expand_dims(voxels, -1)        
#     syn_pcs = uniform_sampling_of_voxels(voxels, n_pc_points, 0, only_lcc=False)
#     return syn_pcs
    
# # print len(all_in_files)

In [70]:
oct_res = 128
n_pc_points = 2048
syn_pcs = np.zeros(shape=(len(all_in_files), n_pc_points, 3))
  
for i, ot_file in enumerate(all_in_files):
    ot_struct, _ = import_ot(ot_file)
    voxels = octree_to_voxel_grid(ot_struct, oct_res).astype(np.bool)
    voxels = np.expand_dims(voxels, 0)        
    syn_pcs[i] = uniform_sampling_of_voxels(voxels, n_pc_points, 0, only_lcc=False)[0]    

In [89]:
ls /orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/gmm/gmm_emd_rifle/

bneck_128_full_32_gaussians.npz


In [99]:
osp.join(top_out_dir, 'bneck_256_full_32_gaussians')

'/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/octree_based/gmm/bneck_256_full_32_gaussians'

In [100]:
np.savez(osp.join(top_out_dir, '2048_pts_bneck_256_full_32_gaussians'), syn_pcs)

In [ ]:
n_threads = 26
def ot_pc_sampler(ot_file):
    oct_res = 128
    n_pc_points = 2048
    ot_struct, _ = import_ot(ot_file)
    voxels = octree_to_voxel_grid(ot_struct, oct_res).astype(np.bool)
    voxels = np.expand_dims(voxels, 0)
    return uniform_sampling_of_voxels(voxels, n_pc_points, 0, only_lcc=False)[0]

n_pc_points = 2048
syn_pcs = np.zeros(shape=(len(all_in_files), n_pc_points, 3))
pool = Pool(n_threads)

for i, data in enumerate(pool.imap(ot_pc_sampler, all_in_files)):
    syn_pcs[i] = data
    
pool.close()
pool.join()